# Random Forest Regression for Harmful Algal Bloom (HAB) Dataset

This script implements and evaluates a time series Random Forest regression workflow to predict HAB using environmental predictors. The analysis includes time-series data splitting, hyperparameter tuning, cross-validation, and feature importance analysis.

## 1. Import Required Libraries
Import all necessary libraries for machine learning, data manipulation, and model evaluation.

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
import numpy as np

## 2. Data Loading and Preparation
Load the dataset and examine its structure. The dataset contains environmental variables that may influence HAB abundance.

In [3]:
# 2.1 Data Loading
# Load the dataset for lake Okeecobee as an example
data = pd.read_excel('Okeechobee.xlsx', header=0)

# Display basic information about the dataset
print(f"Dataset size: {len(data)} samples")
print(f"Dataset shape: {data.shape}")
print("\nFirst few rows:")
print(data.head())


# 2.1 Separate the features (environmental variables) from the target variable (HAB abundance).
# Exclude non-predictive columns like Time and Lake identifiers.
X = data.drop(columns=['Time','CyanHAB','Lake'], axis=1)
y = data['CyanHAB']

print(f"Dataset size: {len(data)} samples")
print(f"Number of features: {X.shape[1]}")
print(f"\nFeature columns: {list(X.columns)}")
print(f"Target variable: CyanHAB")

Dataset size: 85 samples
Dataset shape: (85, 13)

First few rows:
        Time  Solar Rad   Air Temp   Precpt  Wind Speed  Water Temp  \
0 2016-04-01      609.0  23.398667   16.774    5.111161      24.366   
1 2016-05-01      627.0  25.310000   95.484    4.405007      26.042   
2 2016-06-01      552.0  27.379000  139.785    4.269408      29.262   
3 2016-07-01      602.0  28.761000   63.011    3.643123      31.428   
4 2016-08-01      527.0  28.104000  172.473    4.420653      30.567   

   Turbidity      DO       TN      TP   Chl-a        CyanHAB        Lake  
0     49.475  8.4325  1.63500  0.0600  10.290   19910.620328  Okeechobee  
1     39.100  7.8525  1.36250  0.0270  14.265   21639.256122  Okeechobee  
2     23.025  8.3275  1.52250  0.0675  38.300  313020.993949  Okeechobee  
3     18.875  6.9725  1.28250  0.0685  34.050  196983.234522  Okeechobee  
4     15.150  7.3975  1.14575  0.0450  36.930   74421.112524  Okeechobee  
Dataset size: 85 samples
Number of features: 10

Feature 

## 3. Time-Based Train-Test Split
For time series data, it's crucial to preserve temporal order. We use the most recent 30% of data for testing to evaluate how well the model predicts future HAB abundance.

In [5]:
# Splits the data chronologically: first 70% for training, last 30% for testing—preserving time order.
split_idx = int(len(data) * 0.7)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"Training set size: {len(X_train)} samples ({len(X_train)/len(data)*100:.1f}%)")
print(f"Test set size: {len(X_test)} samples ({len(X_test)/len(data)*100:.1f}%)")
print(f"Training period: Index {0} to {split_idx-1}")
print(f"Test period: Index {split_idx} to {len(data)-1}")

Training set size: 59 samples (69.4%)
Test set size: 26 samples (30.6%)
Training period: Index 0 to 58
Test period: Index 59 to 84


## 4. Cross-Validation Setup
Set up TimeSeriesSplit for cross-validation, which respects the temporal nature of the data by only using past data to predict future values.

In [7]:
# User can chnge the number of split
tscv = TimeSeriesSplit(n_splits=3)

print("Time Series Cross-Validation Setup:")
print(f"Number of CV folds: {tscv.n_splits}")
print("Each fold uses progressively more training data to predict the next time period")

Time Series Cross-Validation Setup:
Number of CV folds: 3
Each fold uses progressively more training data to predict the next time period


## 5. Hyperparameter Grid Definition
Define a simplified parameter grid to reduce overfitting risk while still exploring important hyperparameters for Random Forest.

In [9]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [5, 10],
    'max_features': ['sqrt', 0.5],
    'bootstrap': [True]  # Keep bootstrap for variance reduction
}

print("Hyperparameter Grid:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")
    
total_combinations = np.prod([len(v) for v in param_grid.values()])
print(f"\nTotal parameter combinations: {total_combinations}")

Hyperparameter Grid:
  n_estimators: [50, 100]
  max_depth: [10, 20, None]
  min_samples_split: [2, 5]
  min_samples_leaf: [5, 10]
  max_features: ['sqrt', 0.5]
  bootstrap: [True]

Total parameter combinations: 48


## 6. Model Training and Hyperparameter Tuning
Perform grid search with time series cross-validation to find the best hyperparameters for predicting HAB abundance.

In [11]:
# Hyperparameter Tuning with GridSearchCV
model = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # MSE often better for regression
    cv=tscv,  # Use TimeSeriesSplit instead of KFold
    n_jobs=-1,
    verbose=1
)

print("Starting hyperparameter tuning...")

# Perform the grid search on the training set
grid_search.fit(X_train, y_train)

# Best hyperparameters and model
best_model = grid_search.best_estimator_
print("\n" + "="*50)
print("HYPERPARAMETER TUNING RESULTS")
print("="*50)
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score (neg_MSE):", grid_search.best_score_)

Starting hyperparameter tuning...
Fitting 3 folds for each of 48 candidates, totalling 144 fits

HYPERPARAMETER TUNING RESULTS
Best Parameters: {'bootstrap': True, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 100}
Best CV Score (neg_MSE): -1165521863.2197201


## 7. Model Evaluation on Test Set
Evaluate the optimized model on both training and test sets to assess performance and check for overfitting.

In [13]:
y_test_pred = best_model.predict(X_test)
y_train_pred = best_model.predict(X_train)

## 8. Feature Importance Analysis
Analyze which environmental variables are most important for predicting HAB abundance.

In [15]:
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n" + "="*50)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*50)
print("Top 10 Most Important Features:")
print(feature_importance.head(10))

# Save feature importance to Excel file
feature_importance.to_excel('Okeechobee_feature_importance.xlsx', index=False)


FEATURE IMPORTANCE ANALYSIS
Top 10 Most Important Features:
      feature  importance
4  Water Temp    0.239767
1    Air Temp    0.164498
9       Chl-a    0.126645
5   Turbidity    0.122908
2      Precpt    0.113832
6          DO    0.072925
8          TP    0.054559
0   Solar Rad    0.052361
7          TN    0.038220
3  Wind Speed    0.014285


## 9. Save The Result
Store the prediction results for further analysis (multiple matrix: R2, RMSE, NSE etc).

In [17]:
df_test = pd.DataFrame({'Actual': y_test, 'Predicted': y_test_pred})
df_train = pd.DataFrame({'Actual': y_train, 'Predicted': y_train_pred})

# Save the result into Excel file
df_test.to_excel('Okeechobee_test.xlsx', index=False)
df_train.to_excel('Okeechobee_train.xlsx', index=False)